In [1]:
from datasets import Dataset

ds = Dataset.from_parquet("../piidd/data_generation/mixtral-v4-200.pq")
ds

Dataset({
    features: ['id', 'essay', 'prompt', 'names', 'famous_person', 'bio'],
    num_rows: 200
})

In [2]:
import re

def highlight_tokens_html(text, names, famous):
    """
    Generates HTML with names highlighted in yellow and famous names highlighted in green.
    """
    # Import the re module for regex operations

    # Prepare the HTML style
    style = "<style>.yellow { background-color: yellow; } .green { background-color: green; } .red { background-color: red; }</style>"

    # Initialize the HTML with the style tag
    highlighted_html = style + text

    # Function to escape special characters in names for regex
    def escape_names(names):
        return "|".join(re.escape(name) for name in names.split("|") if name)

    # Highlight famous names in green
    if famous:
        famous_pattern = r"\b(" + escape_names(famous) + r")\b"
        highlighted_html = re.sub(famous_pattern, r"<span class='green'>\1</span>", highlighted_html)

    # Highlight other names in yellow
    if names:
        names_pattern = r"\b(" + escape_names(names) + r")\b"
        highlighted_html = re.sub(names_pattern, r"<span class='yellow'>\1</span>", highlighted_html)

    # Highlight "URL" in red
    url_pattern = r"(URL)"
    highlighted_html = re.sub(url_pattern, r"<span class='red'>\1</span>", highlighted_html)

    # Replace newlines with <br> for HTML display
    highlighted_html = highlighted_html.replace("\n", "<br>")

    return highlighted_html

In [3]:
from IPython.display import HTML

x = ds.shuffle()[0]

print("Names:", x["names"])
print("Famous:", x["famous_person"])

HTML(highlight_tokens_html(x["essay"], x["names"], x["famous_person"]))

Names: Lund
Famous: 


In [29]:
print(x["essay"])


Introduction - Problem Identification:

In the rapidly evolving world of Artificial Intelligence (AI), data management and analysis are paramount to success. As a Data Science Engineer, I have faced numerous challenges concerning data quality, accuracy, and accessibility. However, one complex problem that stood out was reducing the dimensionality of high-dimensional datasets while preserving essential information and minimizing information loss. This problem is particularly significant in the AI industry since high-dimensional datasets are prevalent, and effective methods for dimensionality reduction are crucial for streamlining machine learning algorithms, enhancing predictive modeling, and improving overall model performance [1].

Choice of Method or Strategy:

To tackle this complex problem, I chose Principal Component Analysis (PCA) as my method. I based my decision on PCA's wide acceptance and proven success in reducing dimensionality in various domains, including image processin

In [526]:
def remove_duplicates(x):
    url_locations = []
    to_delete = []

    for match in re.finditer(".?URL.?", x):

        inner_matched = False
        for u in url_locations:
            if 0 < match.start() - u < 10:
                x = x[:match.start()] + x[match.end():]
                inner_matched = True
                break
        url_locations.append(match.start())
        
        if inner_matched:
            return remove_duplicates(x)
        
    return x


def remove_word_count(x):

    if "\nWord count:" in x:
        x = x[:x.index("\nWord count:")]
    if "\nWord Count:" in x:
        x = x[:x.index("\nWord Count:")]

    if "\n(Word Count:" in x:
        x = x[:x.index("\nNote:")]

    # (1092 words)

    return x

def remove_note(x):
    if "\nNote:" in x:
        x = x[:x.index("\nNote:")]

    if "\n(Note:" in x:
        x = x[:x.index("(Note:")]

    if "\n[Note:" in x:
        x = x[:x.index("[Note:")]

    return x

def clean_URL(x):

    patterns = [
        r"\nURL: \[[^\n]+\]\n\n",
        r"\n\(URL: [^\n]+\n\n",
        r"\nPersonal URL: [^\n]+\n\n",
        r"\nPersonal URL: [^\n]+$",
    ]
    if "\nURL: [\n\n" in x:
        pass

    pattern = 
    if "\n(URL:.*\n\n":
        pass
    if "\n[URL:.*\n\n":
        pass
    
    pattern = 
    if "\nPersonal URL: [https://www.linkedin.com/in/[YourName]]":
        pass

def clean_essays(x):
    essay = x["essay"]
    names = x["names"]
    famous = x["famous_person"]

    # url_matches = re.finditer(r"\n[a-zA-Z\(\[ ]*URL[a-zA-Z\(\[\)\] :\.]*\n", essay)

    replaced = re.sub(r"\n[a-zA-Z\(\[ ]*URL[a-zA-Z\(\[\)\] :\./0-9'\"]*[\n\b]", " <URL> ", essay)

    url_locations = list(re.finditer("<URL>", replaced))
    other_urls = list(re.finditer("[URL]", replaced))

    hyperlink_pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*))'
    hyperlinks = list(re.finditer(hyperlink_pattern, replaced))

    other_hyperlinks = list(re.finditer(r'www\.[a-zA-Z0-9\-\.]+\.[a-z]{2,6}', replaced))

    to_delete = []

    for hyperlink in [*hyperlinks, *other_hyperlinks]:
        start = hyperlink.start()
        end = hyperlink.end()
        
        for url_location in url_locations:
            if 0 < start - url_location.end() < 5 or 0 < end - url_location.start() < 5:
                to_delete.append(hyperlink)
        
        for other_url in other_urls:
            if 0 < start - other_url.start() < 5 or 0 < end - other_url.end() < 5:
                to_delete.append(hyperlink)


    for hyperlink in to_delete:
        replaced = re.sub(re.escape(hyperlink.group(0)), " <URL> ", replaced)

    if "\nWord count:" in replaced:
        replaced = replaced[:replaced.index("\nWord count:")]
    if "\nWord Count:" in replaced:
        replaced = replaced[:replaced.index("\nWord Count:")]

    if "\nNote:" in replaced:
        replaced = replaced[:replaced.index("\nNote:")]

    replaced = remove_duplicates(replaced)


    return {
        "replaced": replaced
    }
    

ds = ds.map(clean_essays)

SyntaxError: unterminated string literal (detected at line 38) (1639798326.py, line 38)

In [362]:
x = all_ds.shuffle()[0]

        # #personalurl#
# [URL, remove brackets]
# [insert personal URL].
# URL removed
# [Insert Personal URL Here]

pattern =  "|".join([
        r"\nURL: [^\n]+\]\n\n", # URL: [https://www.linkedin.com/in/[YourName]]\n\n
        r"\nURL: \[[^\n]+$", # URL: [https://www.linkedin.com/in/[YourName]]
        r"\n[\(\[]URL: [^\n]+\n\n", # (URL: [https://www.linkedin.com/in/[YourName]]\n\n
        r"\nPersonal URL: [^\n]+\n\n", # Personal URL: https://www.linkedin.com/in/[YourName]\n\n
        r"\nPersonal URL: [^\n]+$", # Personal URL: https://www.linkedin.com/in/[YourName]
        r"\n ?\[URL\][\[\(][^\n]+\n\n", # [URL](http://www.example.com)\n\n
        r"\n\[URL\][\[\(][^\n]+$", # [URL](http://www.example.com)
        r"\n\[URL[^\n]+\]\n", # [URL: http://www.example.com]\n\n
        r"\n\[URL[^\n]+$", # [URL: http://www.example.com]
        r"\n\[URL\][^\n]+\n", # [URL: http://www.example.com]\n
    ])

count = 0
while len(re.findall(pattern, x["essay"])) == 0 and "URL" not in x["essay"]:
    x = all_ds.shuffle()[0]
    count += 1
    if count > 1000: break

essay = re.sub(pattern, " <URL> ", x["essay"])

print("Names:", x["names"])
print("Famous:", x["famous_person"])
print(re.findall(pattern, x["essay"]))

HTML(highlight_tokens_html(essay, x["names"], x["famous_person"]))

Names: Apollo
Famous: 
['\n[URL: https://www.systems-thinking.org/]']


In [314]:
re.findall(r"\n ?\[URL\][\[\(][^\n]+\n\n", x["essay"])

['\n [URL](https://www.telecom-experts.com/blog/case-study-telecom-equipment-specialist-tackles-network-challenge)\n\n']

In [372]:
[URL] www.linkedin.com/in/dedicated-paint-technician/

["\nThe Complex Problem of Brake Judder and Its Solution Through Applied Methodology\n\nAs an Automotive Brake Specialist, I have encountered numerous complex problems related to automotive brake systems. One prevalent issue that often challenges even the most experienced specialists is brake judder, which can compromise the safety and performance of vehicles. This essay will describe my application of a specific methodology to tackle this complex problem and critically evaluate its effectiveness.\n\nBrake judder manifests as a vibration or pulsating sensation when applying the brakes and is typically caused by unevenly worn brake discs or pads. The problem can worsen over time, leading to a significant decrease in braking performance and safety. Given its potential impact on both drivers and pedestrians, it is crucial to address brake judder effectively and efficiently.\n\nFor this complex problem, I chose to apply the Plan-Do-Check-Act (PDCA) methodology, which is a cyclical and iter

In [312]:
x["essay"]

"\nThe Challenge: A Multifaceted Telecom Network Dilemma\n\nAs a Telecommunications Equipment Specialist, I faced a complex challenge when a major client approached me with an issue. They were experiencing frequent network downtime, affecting their business operations and customer satisfaction. The client's telecom network spanned multiple buildings with a mix of old and new equipment, making the issue difficult to diagnose and resolve.\n\nSelecting the Right Tool: Analytics and Expertise\n\nI decided to deploy network analytics software as my primary tool, supplemented by my expertise in telecom systems and problem-solving skills. Analytics could quickly process vast amounts of data and reveal patterns, trends, and potential causes of the downtime. Moreover, the software would help me efficiently monitor and manage the network, maximizing uptime and minimizing service disruptions.\n\nChoosing this tool over alternative options, like manual diagnostics or reactive maintenance, was a st

In [364]:
m

[<re.Match object; span=(0, 5), match='[URL]'>]

In [269]:
hyperlink_pattern = r'(https?:\/\/)?(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*)'

hyperlinks = list(re.finditer(hyperlink_pattern, x["essay"]))

In [270]:
len(hyperlinks)

2

In [305]:
hyperlinks

[<re.Match object; span=(5659, 5716), match='www.financialanalystexpert.com/blog/stock-splits->,
 <re.Match object; span=(5718, 5782), match='http://www.financialanalystexpert.com/blog/stock->]

In [420]:
x["essay"]

'\nIntroduction:\n\nAs an electrical assembler in the manufacturing industry, I encounter a myriad of complex issues on a daily basis. From rewiring malfunctioning systems to interpreting complex engineering blueprints, my role demands a high level of problem-solving acumen. However, one issue stands out as particularly challenging and significant: the optimization of electrical assembly lines to increase efficiency and productivity while maintaining quality control.\n\nStrategy Choice:\n\nTo tackle this complex issue, I chose a strategy known as the DMAIC (Define, Measure, Analyze, Improve, Control) approach. This strategy is commonly used in Six Sigma methodology and is effective in solving complex problems by breaking them down into manageable phases. I chose this strategy because of its comprehensive and analytical nature, as well as its emphasis on data-driven decision-making. Alternative strategies such as the PDCA (Plan, Do, Check, Act) cycle and the 8D problem-solving process w

In [4]:
from pathlib import Path
from datasets import concatenate_datasets


all_ds = concatenate_datasets([Dataset.from_parquet(str(x)) for x in Path("../piidd/data_generation/").glob("mixtral-*.pq")])

In [427]:
all_ds

Dataset({
    features: ['id', 'essay', 'prompt', 'names', 'famous_person', 'bio'],
    num_rows: 5800
})

In [428]:
len(set(all_ds["id"])), len([x for x in all_ds["essay"] if "URL" in x])

(5800, 5413)

In [431]:
essays = {i: (e, n, f) for i, e, n, f in zip(all_ds["id"], all_ds["essay"], all_ds["names"], all_ds["famous_person"]) if "URL" in e}

In [434]:
good_ids = set()
bad_ids = set()

In [436]:
bad_ids

set()

In [437]:
good_ids

{'kbbrnhmk'}

In [439]:
import gradio as gr
import datetime
import pickle



def bad_click(id):
    bad_ids.add(id)

    with open("bad_ids.pkl", "wb") as f:
        pickle.dump(bad_ids, f)

    return next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)

def good_click(id):

    good_ids.add(id)

    with open("good_ids.pkl", "wb") as f:
        pickle.dump(good_ids, f)

    return next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)


def show_html(id):
    e, n, f = essays[id]
    return highlight_tokens_html(e, n, f)

first_id = next(x for x in essays.keys() if x not in good_ids and x not in bad_ids)

with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            bad = gr.Button("Bad")
            good = gr.Button("Good")
        id = gr.Textbox(value=first_id)
        html = gr.HTML(value=show_html(first_id))

    id.change(fn=show_html, inputs=[id], outputs=[html])
    bad.click(fn=bad_click, inputs=[id], outputs=[id])
    good.click(fn=good_click, inputs=[id], outputs=[id])


demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
